# M/M/1 Simulator

Packages

In [26]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from numpy import *
import os
import random

Manage queue- find the next available costumer (FCFS policy)

In [27]:
#for single class setting
def find_next_index(df,time):
    df=df[df['scheduled'] == 0]
    if df.empty: # for self-use  
        return -1
    else:
        min_time=df['arrival'].min()
        df=df[df['arrival'] == min_time]
        return df.iloc[0,0]

Calculate queue length

In [28]:
# for each setting   
def q_len(df,time):
    df_1=df[df['scheduled'] == 0]
    df_1=df_1[df_1['arrival'] <= time]
    if df_1.empty:
        return 0
    else:
        return len(df_1)

In [ ]:
Calculate waiting time

In [29]:
def calc_wt(df,time):
    df_1 = df[df['arrival'] <= time]
    df_1 = df_1[df_1['service'] > time]
    if df_1.empty:
        return 0
    else:
        en_max = df_1['arrival'].max()
        x = df_1[df_1['arrival'] == en_max ]
        x = x.reset_index(drop=True)
        wt = (x.at[0, 'service']) - time
        return wt  

Create event log

In [30]:
def create_event_log(sample_arrivals, sample_services, class_flag ): # class_flag is binary: 1=multi class, 0=single class
    #Data-frame intiation
    df_sim = pd.DataFrame(sample_arrivals,columns = ['arrival'])
    df_sim.reset_index(inplace=True)
    df_sim['waiting'] = 0.0
    df_sim['service'] = 0.0
    df_sim['execution'] = 0.0
    df_sim['completion'] = 0.0
    df_sim['sojourn_time'] = 0.0
    if class_flag == 1: 
        sample_class = list(np.random.choice(class_num , len(df_sim), class_distribution))
        df_sim['class'] = sample_class
    df_sim['scheduled'] = 0
    df_sim['order'] = 0
    df_sim['busy'] = 0
    df_sim['queue_length'] = 0

    t = df_sim['arrival'].min() #The time when the first sutomer starts getting service
    i = 0

    q_len_seq = [] #list for the plot
    time_seq = [] #list for the plot
    list_busy = []
    bp_len = 0
    while i < len(df_sim):
        if class_flag == 0: #single class
            j = find_next_index(df_sim, t) #find the next available costumer
        else: #multi-class
            j = find_next_index_class(df_sim, t) #find the next available costumer    
        if j != -1:
            if df_sim.at[j, 'arrival'] <= t: #the customer wait for service
                df_sim.at[j, 'service'] = t
                bp_len +=1
                df_sim.at[j, 'busy'] = 1
            else: # server wait to customer
                df_sim.at[j, 'service'] = df_sim.at[j, 'arrival']
                if bp_len > 0:
                    list_busy.append(bp_len)
                bp_len = 0
            df_sim.at[j, 'completion'] = df_sim.at[j,'service'] + sample_services[j]
            df_sim.at[j,'execution'] = sample_services[j]
            df_sim.at[j, 'scheduled'] = 1
            df_sim.at[j, 'queue_length'] = q_len(df_sim,t)
            time_seq.append(df_sim.at[j, 'service'])
            q_len_seq.append(df_sim.at[j, 'queue_length'])
            t= df_sim.at[j, 'completion']
            df_sim.at[j, 'order'] = i
            i = i+1
        else: 
            print('eroor')
    
    if bp_len > 0:
        list_busy.append(bp_len) 
        
    for i in range(len(df_sim)):
        df_sim.at[i, 'sojourn_time'] = df_sim.at[i,'completion'] - df_sim.at[i,'arrival']
        df_sim.at[i, 'wt'] = calc_wt(df_sim, df_sim.at[i, 'service'] )
    df_sim.at[0, 'busy'] = 0    
    return df_sim, q_len_seq, time_seq, list_busy  

This fuction get simulation in Dataframe format and return Dataframes that include all the  busy-periods 

In [18]:
def create_busy_periods(df, j):
     list_busy = []
    period = 0
    k = 0 #local lower-bound
    l = 0 #local upper-bound

    for i in range(len(df)):
        if df.at[i,'busy'] == 1:
            if period == 0 and df.at[i-1,'busy'] == 0:
                k = i-1
                l = i
                period += 1
            else: 
                l = i
            period += 1
        else:
            if period >15:
                l = l+1
                temp = df[k:l]
                temp = temp.reset_index(drop=True) 
                min_time = temp['service'].min()
                temp['arrival'] = temp['arrival'].map(lambda x: x - min_time )
                temp['service'] = temp['service'].map(lambda x: x - min_time )
                temp['completion'] = temp['completion'].map(lambda x: x - min_time )
                temp.to_csv(r'simulations/busy_periods/bp_%s.csv' %j, index = False)
                j += 1

            period = 0
    if period > 15:
        l = l+1
        temp = df[k:l]
        temp = temp.reset_index(drop=True) 
        min_time = temp['arrival'].min()
        temp['arrival'] = temp['arrival'].map(lambda x: x - min_time )
        temp['service'] = temp['service'].map(lambda x: x - min_time )
        temp['completion'] = temp['completion'].map(lambda x: x - min_time )
        temp.to_csv(r'simulations/busy_periods/bp_%s.csv' %j, index = False)
        j += 1
        
    return j
           

The simulator code:

In [ ]:
num_of_arrivals = 55000
intensity_arrivals = [0.84, 0.86, 0.88, 0.9, 0.92] # 
intensity_services = 1 

j = 0 # indicator for the number of each busy period output.
for dist in range(len(intensity_arrivals)):
    scale_parameter_arrivals= 1/intensity_arrivals[dist]
    scale_parameter_services= 1/intensity_services

    sample_arrivals = list(np.random.exponential( scale=float(scale_parameter_arrivals), size=num_of_arrivals))
    sample_arrivals[0] = 0.0

    f= open("simulations/sim(M_M_1)_lambda_%s.txt" %(dist),"w+")
    f.write("number of arrivals: %s\n" % (num_of_arrivals))
    f.write("arrivals rate: %s  \r\n" % (intensity_arrivals[dist]))
    f.write("service rate: %s \r\n" % (intensity_services))


    for i in range(1, num_of_arrivals):
        sample_arrivals[i] = sample_arrivals[i] + sample_arrivals[i-1]   

    x_min = min(sample_arrivals)
    if x_min < 0:
        sample_arrivals = [x - x_min for x in sample_arrivals ]
        
    sample_arrivals.sort()
        
    #sample executions
    sample_services = list(np.random.exponential( scale=float(scale_parameter_services), size=num_of_arrivals))    

    df, q_len_seq, time_seq, list_busy = create_event_log(sample_arrivals, sample_services, class_flag = 0 )
    df.to_csv(r'simulations/sim_%s.csv' %(dist) , index = False)


    f.write('Lengths of busy-periods: %s\n' %(list_busy))
    avg = (sum(list_busy)/len(list_busy))
    f.write('Average length of busy-period: %d\n' %(avg))
    maxi = max(list_busy)
    f.write('Max length of busy-period: %d\r\n' %(maxi))

    # plot queue length\time
    plt.figure()
    plt.plot(time_seq , q_len_seq  )
    plt.axis([0, df['completion'].max(), 0, df['queue_length'].max()])
    plt.ylabel('Number of cases in queue')
    plt.xlabel('Time')
    plt.title('Queue length')
    plt.savefig('simulations/sim(M_M_1)_%s.png' %(dist))

    f.close() 
    
    j = create_busy_periods(df, j)

